In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
df = pd.read_csv("kw_email_use_selenium.csv")

url_list = df['agent_url'].unique()
def get_driver():
    options = webdriver.FirefoxOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Firefox(options=options)
    return driver


def collect_agent_contact_details(driver):
    try:
        agent_info = driver.find_element_by_class_name("AgentInformation")
    except:
        agent_info = ""
    try:
        info_container = agent_info.find_elements_by_class_name("AgentInformation__factBody")
    except:
        info_container = ""
    try:
        agent_email = [email.text for email in info_container if email.get_attribute("aria-label") == 'Agent E-mail'][0]
    except:
        agent_email = ""
    try:
        mobile_number = driver.find_element_by_class_name("AgentInformation__phoneMobileNumber").text
    except:
        mobile_number = ""
    try:
        office_number = driver.find_element_by_class_name("AgentInformation__phoneOfficeNumber").text
    except:
        office_number = ""
    try:
        agent_website = [i.text for i in info_container if i.get_attribute("aria-label") == 'Agent Website'][0]
    except:
        agent_website = ""
    contact_dict = {"agent_email":agent_email,"mobile_number":mobile_number,"office_number":office_number,'agent_website':agent_website }
    return contact_dict


def get_agent_details(driver):
    try:
        agent_licenses = driver.find_element_by_class_name("AgentContent__licenses").text
    except:
        agent_licenses = ""
    try:
        agent_bio = driver.find_element_by_class_name("AgentContent__bio").text
    except:
        agent_bio = ""
    try:
        serviceAreas = driver.find_element_by_class_name("AgentContent__serviceAreas").text
    except:
        serviceAreas = ""
    try:
        agent_team_name = driver.find_element_by_class_name("AgentContent__teamName").text
    except:
        agent_team_name = ""
    try:
        agent_team_info_ = driver.find_element_by_class_name("AgentContent__teamText").text.split("\n")
        agent_team_info = ", ".join(agent_team_info_)
    except:
        agent_team_info = ""
    try:
        logo_url = driver.find_element_by_class_name('AgentContent__teamAvatar').find_element_by_class_name("KWImage__image").get_attribute("src")
    except:
        logo_url = ""
    agent_details = {"agent_licenses":agent_licenses, "agent_bio":agent_bio, "serviceAreas":serviceAreas,"agent_team_name":agent_team_name,
                    "agent_team_info":agent_team_info,'logo_url':logo_url}
    return agent_details


def social_media_details(driver):
    try:
        social_media_container = driver.find_element_by_class_name("AgentInformation__socialMedia").find_elements_by_class_name("link")
    except:
        social_media_container = []
    social_media_links = [social_media.get_attribute("href") for social_media in social_media_container]
    try:
        facebook = [facebook for facebook in social_media_links if 'facebook' in facebook][0]
    except:
        facebook = ""
    try:
        instagram = [instagram for instagram in social_media_links if 'instagram' in instagram][0]
    except:
        instagram = ""
    try:
        twitter = [twitter for twitter in social_media_links if 'twitter' in twitter][0]
    except:
        twitter = ""
    try:
        linkedin = [linkedin for linkedin in social_media_links if 'linkedin' in linkedin][0]
    except:
        linkedin = ""
    social_media_dict = {"facebook":facebook,"instagram":instagram,"twitter":twitter,"linkedin":linkedin}
    social_media_links_str = "|".join(social_media_links)
    return social_media_dict,social_media_links_str


def collect_other_info(driver):
    container = driver.find_elements_by_class_name("AgentContent__section")
    try:
        Market_Cente = [i for i in container if 'Market Center' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Market_Cente = ""
    try:
        Languages = [i for i in container if 'Languages' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Languages = ""
    try:
        Specialties_designation = [i for i in container if 'Specialties and Designations' in i.text][0].find_element_by_class_name("AgentContent__sectionText").text
    except:
        Specialties_designation = ""
    other_info = {"Market_Cente":Market_Cente, "Languages":Languages, "Specialties_Designations":Specialties_designation}
    return other_info

def wait_to_page_load(driver):
    try:
        w = WebDriverWait(driver, 3)
        w.until(EC.presence_of_element_located((By.CLASS_NAME,"AgentContent__name")))
        print("Page load happened")
    except Exception as e:
        print("Timeout happened no page load")
    return driver

url_count = 48000
driver_count = 0
driver = get_driver()
for url in url_list[url_count:49000]:
    print(url)
    if driver_count >=100:
        print("need to close driver")
        driver.quit()
        driver_count = 0
        driver = get_driver()
        print("new driver initialize****************")
    driver.get(url)
    driver = wait_to_page_load(driver)
    try:
        agent_name = driver.find_element_by_class_name("AgentContent__name").text
    except:
        agent_name = ""
    try:
        time.sleep(0.5)
        profile_url = driver.find_element_by_class_name("AvatarImage__bg").get_attribute("style").replace('background-image: url("','').replace('");','')
    except:
        profile_url = ""
    try:
        tag = driver.find_element_by_class_name("pill").text
    except:
        tag = ""
    try:
        role = driver.find_element_by_class_name("AgentContent__team").text
    except:
        role = ""
    try:
        location = driver.find_element_by_class_name("AgentContent__location").text
    except:
        location = ""
    contact_dict = collect_agent_contact_details(driver)
    agent_details = get_agent_details(driver)
    social_media_dict,social_media_links_str = social_media_details(driver)
    other_info = collect_other_info(driver)
    data_dict = {'agent_url':url,"agent_name":agent_name, 'agent_role':role,'location':location,"agent_dp":profile_url, "tag":tag,"agent_email":contact_dict['agent_email'], "mobile_number":contact_dict['mobile_number'],"office_number":contact_dict['office_number'],'agent_license':agent_details['agent_licenses'], 'agent_bio':agent_details['agent_bio'],'serviceAreas':agent_details['serviceAreas'],'office_name':agent_details['agent_team_name'],'office_address':agent_details['agent_team_info'],'logo_url':agent_details['logo_url'],'facebook':social_media_dict['facebook'], 'instagram':social_media_dict['instagram'],'twitter':social_media_dict['twitter'],'linkedin':social_media_dict['linkedin'],'Market_Cente':other_info['Market_Cente'],'agent_language':other_info['Languages'],'Specialties_Designations':other_info['Specialties_Designations'],'social_media_links_str':social_media_links_str,'agent_website':contact_dict['agent_website']}
    data_df = pd.DataFrame(data_dict,index=[0],columns=['agent_url','agent_name', 'agent_role','location','agent_dp', 'tag','agent_email', 'mobile_number','office_number', 'agent_license', 'agent_bio', 'serviceAreas','office_name', 'office_address','logo_url','facebook', 'instagram', 'twitter', 'linkedin','Market_Cente', 'agent_language', 'Specialties_Designations','social_media_links_str','agent_website'])
    with open("kw_data-emailSelenium_48k_49k.csv",'a',newline='',encoding='utf-8') as f:
        data_df.to_csv(f, mode='a',header=f.tell()==0)
    print(url_count,url)
    url_count+=1
    driver_count+=1
    print("***********************************")
driver.quit()


https://kw.com/agent/UPA-6587385015355367424-4
Page load happened
48000 https://kw.com/agent/UPA-6587385015355367424-4
***********************************
https://kw.com/agent/UPA-6587385077633642497-6
Page load happened
48001 https://kw.com/agent/UPA-6587385077633642497-6
***********************************
https://kw.com/agent/UPA-6721805474485481472-0
Page load happened
48002 https://kw.com/agent/UPA-6721805474485481472-0
***********************************
https://kw.com/agent/UPA-6617836769915326466-2
Page load happened
48003 https://kw.com/agent/UPA-6617836769915326466-2
***********************************
https://kw.com/agent/UPA-6592989721621037062-7
Page load happened
48004 https://kw.com/agent/UPA-6592989721621037062-7
***********************************
https://kw.com/agent/UPA-6772937312505192448-2
Page load happened
48005 https://kw.com/agent/UPA-6772937312505192448-2
***********************************
https://kw.com/agent/UPA-6587385151071854594-4
Page load happened
4800

Page load happened
48053 https://kw.com/agent/UPA-6587385254746005510-3
***********************************
https://kw.com/agent/UPA-6741859276414750720-2
Page load happened
48054 https://kw.com/agent/UPA-6741859276414750720-2
***********************************
https://kw.com/agent/UPA-6587385179969929218-8
Page load happened
48055 https://kw.com/agent/UPA-6587385179969929218-8
***********************************
https://kw.com/agent/UPA-6641742543736438784-5
Page load happened
48056 https://kw.com/agent/UPA-6641742543736438784-5
***********************************
https://kw.com/agent/UPA-6587385064006254593-5
Page load happened
48057 https://kw.com/agent/UPA-6587385064006254593-5
***********************************
https://kw.com/agent/UPA-6587385166250319873-2
Page load happened
48058 https://kw.com/agent/UPA-6587385166250319873-2
***********************************
https://kw.com/agent/UPA-6587385377228812289-2
Page load happened
48059 https://kw.com/agent/UPA-6587385377228812289-

Page load happened
48106 https://kw.com/agent/UPA-6587385309251964935-6
***********************************
https://kw.com/agent/UPA-6587385170520604675-9
Page load happened
48107 https://kw.com/agent/UPA-6587385170520604675-9
***********************************
https://kw.com/agent/UPA-6587385033857835008-5
Page load happened
48108 https://kw.com/agent/UPA-6587385033857835008-5
***********************************
https://kw.com/agent/UPA-6609031537416957955-2
Page load happened
48109 https://kw.com/agent/UPA-6609031537416957955-2
***********************************
https://kw.com/agent/UPA-6651515711527604224-0
Page load happened
48110 https://kw.com/agent/UPA-6651515711527604224-0
***********************************
https://kw.com/agent/UPA-6587385281890418688-5
Page load happened
48111 https://kw.com/agent/UPA-6587385281890418688-5
***********************************
https://kw.com/agent/UPA-6587385185282723842-8
Page load happened
48112 https://kw.com/agent/UPA-6587385185282723842-

Page load happened
48159 https://kw.com/agent/UPA-6766108478037823488-4
***********************************
https://kw.com/agent/UPA-6732390173344653312-5
Page load happened
48160 https://kw.com/agent/UPA-6732390173344653312-5
***********************************
https://kw.com/agent/UPA-6601785102434447364-0
Page load happened
48161 https://kw.com/agent/UPA-6601785102434447364-0
***********************************
https://kw.com/agent/UPA-6587385165543108609-2
Page load happened
48162 https://kw.com/agent/UPA-6587385165543108609-2
***********************************
https://kw.com/agent/UPA-6587385159333072896-9
Page load happened
48163 https://kw.com/agent/UPA-6587385159333072896-9
***********************************
https://kw.com/agent/UPA-6823652592743071744-0
Page load happened
48164 https://kw.com/agent/UPA-6823652592743071744-0
***********************************
https://kw.com/agent/UPA-6831941314899001344-2
Page load happened
48165 https://kw.com/agent/UPA-6831941314899001344-

Page load happened
48212 https://kw.com/agent/UPA-6587385191077449732-2
***********************************
https://kw.com/agent/UPA-6592991040537255936-1
Page load happened
48213 https://kw.com/agent/UPA-6592991040537255936-1
***********************************
https://kw.com/agent/UPA-6587385319708127240-9
Page load happened
48214 https://kw.com/agent/UPA-6587385319708127240-9
***********************************
https://kw.com/agent/UPA-6839777217264734208-5
Page load happened
48215 https://kw.com/agent/UPA-6839777217264734208-5
***********************************
https://kw.com/agent/UPA-6592242008952070149-4
Page load happened
48216 https://kw.com/agent/UPA-6592242008952070149-4
***********************************
https://kw.com/agent/UPA-6742204826245423104-7
Page load happened
48217 https://kw.com/agent/UPA-6742204826245423104-7
***********************************
https://kw.com/agent/UPA-6806619476873932800-9
Timeout happened no page load
48218 https://kw.com/agent/UPA-680661947

...

Timeout happened no page load
48741 https://kw.com/agent/UPA-6587385058847604741-1
***********************************
https://kw.com/agent/UPA-6587385437341577219-6
Page load happened
48742 https://kw.com/agent/UPA-6587385437341577219-6
***********************************
https://kw.com/agent/UPA-6587385202028539911-8
Page load happened
48743 https://kw.com/agent/UPA-6587385202028539911-8
***********************************
https://kw.com/agent/UPA-6734927537945403392-0
Page load happened
48744 https://kw.com/agent/UPA-6734927537945403392-0
***********************************
https://kw.com/agent/UPA-6587385350858309633-5
Page load happened
48745 https://kw.com/agent/UPA-6587385350858309633-5
***********************************
https://kw.com/agent/UPA-6587385293762400264-7
Page load happened
48746 https://kw.com/agent/UPA-6587385293762400264-7
***********************************
https://kw.com/agent/UPA-6587385342461419524-3
Page load happened
48747 https://kw.com/agent/UPA-658738534

Page load happened
48794 https://kw.com/agent/UPA-6762061277857026048-8
***********************************
https://kw.com/agent/UPA-6682380583773356032-3
Page load happened
48795 https://kw.com/agent/UPA-6682380583773356032-3
***********************************
https://kw.com/agent/UPA-6587385260543275014-4
Page load happened
48796 https://kw.com/agent/UPA-6587385260543275014-4
***********************************
https://kw.com/agent/UPA-6816026670715772928-7
Page load happened
48797 https://kw.com/agent/UPA-6816026670715772928-7
***********************************
https://kw.com/agent/UPA-6704456857998512128-5
Page load happened
48798 https://kw.com/agent/UPA-6704456857998512128-5
***********************************
https://kw.com/agent/UPA-6587385355602751492-9
Page load happened
48799 https://kw.com/agent/UPA-6587385355602751492-9
***********************************
https://kw.com/agent/UPA-6779893680436633600-6
need to close driver
new driver initialize****************
Timeout hap

Page load happened
48847 https://kw.com/agent/UPA-6714286826972004352-7
***********************************
https://kw.com/agent/UPA-6592245210675351558-6
Page load happened
48848 https://kw.com/agent/UPA-6592245210675351558-6
***********************************
https://kw.com/agent/UPA-6781238204011401216-3
Timeout happened no page load
48849 https://kw.com/agent/UPA-6781238204011401216-3
***********************************
https://kw.com/agent/UPA-6629058439730655237-8
Page load happened
48850 https://kw.com/agent/UPA-6629058439730655237-8
***********************************
https://kw.com/agent/UPA-6587384978532790279-9
Page load happened
48851 https://kw.com/agent/UPA-6587384978532790279-9
***********************************
https://kw.com/agent/UPA-6793921321287426048-1
Page load happened
48852 https://kw.com/agent/UPA-6793921321287426048-1
***********************************
https://kw.com/agent/UPA-6587385235979169795-5
Page load happened
48853 https://kw.com/agent/UPA-658738523

new driver initialize****************
Page load happened
48900 https://kw.com/agent/UPA-6826219188670570496-6
***********************************
https://kw.com/agent/UPA-6773386044350337024-0
Page load happened
48901 https://kw.com/agent/UPA-6773386044350337024-0
***********************************
https://kw.com/agent/UPA-6813942538659074048-2
Page load happened
48902 https://kw.com/agent/UPA-6813942538659074048-2
***********************************
https://kw.com/agent/UPA-6782777258354442240-1
Page load happened
48903 https://kw.com/agent/UPA-6782777258354442240-1
***********************************
https://kw.com/agent/UPA-6587385168427630595-3
Page load happened
48904 https://kw.com/agent/UPA-6587385168427630595-3
***********************************
https://kw.com/agent/UPA-6831699349140770816-2
Page load happened
48905 https://kw.com/agent/UPA-6831699349140770816-2
***********************************
https://kw.com/agent/UPA-6587385191921569798-0
Page load happened
48906 https:/

Page load happened
48953 https://kw.com/agent/UPA-6636683265597706241-1
***********************************
https://kw.com/agent/UPA-6772207773524357120-3
Page load happened
48954 https://kw.com/agent/UPA-6772207773524357120-3
***********************************
https://kw.com/agent/UPA-6587385340342239237-7
Page load happened
48955 https://kw.com/agent/UPA-6587385340342239237-7
***********************************
https://kw.com/agent/UPA-6587385165471805446-6
Page load happened
48956 https://kw.com/agent/UPA-6587385165471805446-6
***********************************
https://kw.com/agent/UPA-6605415559888470018-8
Page load happened
48957 https://kw.com/agent/UPA-6605415559888470018-8
***********************************
https://kw.com/agent/UPA-6628397655157424133-2
Page load happened
48958 https://kw.com/agent/UPA-6628397655157424133-2
***********************************
https://kw.com/agent/UPA-6823727400849534976-1
Page load happened
48959 https://kw.com/agent/UPA-6823727400849534976-